In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install scikit-learn
# !pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 74.6 MB/s eta 0:00:00


In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 33.4 MB/s eta 0:00:00


In [3]:
import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

In [4]:
import pandas as pd

path = '/content/drive/MyDrive/최종프로젝트/data/Lotion_dataset_clean.csv'

df = pd.read_csv(path, index_col=False)
raw = df.copy()

In [5]:
df.head(3)

,Unnamed: 0,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,0,AHC 온리 포맨 로션 150ml,AHC,"25,000","14,000",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,150ml,모든 피부용,"정제수, 글리세린, 디메치콘, 나이아신아마이드, 사이클로펜타실록산, 세틸알코올, 카...",아빠께서 로션을 열심히 찾고 계시길래 같이 가서 테스트 해보고 구매한 제품입니다 아...
1,1,BRTC 파워 옴므 올인원 솔루션 200ml,비알티씨,"28,000","25,600",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,200ml,모든 피부 타입,"정제수, 부틸렌글라이콜, 글리세린, 에탄올, 나이아신아마이드, 비스-피이지-18메칠...",디자인도 멋있고 고급스러워 남편 선물용으로 딱이겠더라고요 스킨 로션 에센스까지 한번...
2,5,[NEW/리뉴얼] 에스트라 아토베리어365 로션 150ml,에스트라,"33,000","33,000",https://image.oliveyoung.co.kr/uploads/images/...,https://www.oliveyoung.co.kr/store/goods/getGo...,150ml,모든 피부용,"정제수, 부틸렌글라이콜, 글리세린, 다이메티콘, 스쿠알란, 펜타에리스리틸테트라아이소...",너무 기름지지 않고 보습력 좋게 가볍게 바를수있는 로션이에요 호불호없이 사용 가능 ...


In [6]:
reviews = df['review']
reviews

0     아빠께서 로션을 열심히 찾고 계시길래 같이 가서 테스트 해보고 구매한 제품입니다 아...
1     디자인도 멋있고 고급스러워 남편 선물용으로 딱이겠더라고요 스킨 로션 에센스까지 한번...
2     너무 기름지지 않고 보습력 좋게 가볍게 바를수있는 로션이에요 호불호없이 사용 가능 ...
3     진정이 되는지도 잘 모르겠고 그냥 무난한데 세검이 유명한 라인이라 로션 시도한건데 ...
4     남자친구 주려고 샀어요 남자친구가 만족하면서 쓰는 거 같아요 ㅋㅋ 저도 써봤는데 좋...
                            ...                        
62    촉촉하고 무난하게 쓰기 좋아요 무향에 가까워서 남자도 쓰기 좋습니다 이거만한게 없습...
63    수부지 각질부각에 트러블까지 있으신분들 진짜 강추입니다 왜냐면 제 피부가 그랬거든요...
64    기본 밀크보다 더 촉촉한데 부드럽게 흡수됩니다 그리고 번들거리지도 않습니다 부드럽고...
65    여향갔다가 하도 유명해서 사봤는데 너무 순하고 보습력도 좋아서 또 재구매해서 쓰고있...
66    엄마가 한율 에센스도 사용중이고 만족해하세요 에멀젼 역시 만족해하시네요 재구매 예정...
Name: review, Length: 67, dtype: object

In [ ]:
reviews = []

for r in df['review']:
    reviews.append([r])

불용어

In [7]:
path = '/content/drive/MyDrive/최종프로젝트/data/stopword_list.xlsx'

stop_words = pd.read_excel(path)


In [8]:
stop_words = stop_words['stopword'].tolist()

In [9]:
stop_words.append('아앙')
print(stop_words)

['가까스로', '가량', '가령', '가민', '가민커넥트', '가슴', '가슴살', '각', '가지', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개발', '개발자', '개별로', '개월', '개의치않고', '거기', '거니와', '거바', '거의', '건데', '건지', '걸까요', '걸로', '것', '것과 같이', '것들', '게다가', '게요', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '경우', '계속', '고감', '고로', '곧', '고려하면', '고민', '고생', '고요', '공동으로', '과', '과연', '관계가 있다', '관련', '관련이 있다', '관리자', '관한', '관하여', '관해서는', '구', '구글플레이스토어', '구체적으로', '구토하다', '귀차 니즘', '귀차니즘', '그', '그거', '그건', '그걸', '그걸로', '그것', '그게', '그곳', '그날', '그동안', '그들', '그때', '그래', '그래도', '그래서', '그램', '그러나', '그러니', '그러니까', '그러면', '그러므로', '그러한즉', '그럼', '그렇게 함으로써', '그렇지', '그렇지만', '그리고', '그리하여', '그에 따르는', '그위에', '그인', '그저', '그중에서', '근거로', '근거하여', '금요일', '기능', '기대여', '기모', '기점으로', '기준으로', '기타', '기프', '까닭으로', '까악', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나름', '나머지는', '나세', '나용', '나이키', '나중', '난리', '남들', '남자', '남짓', '내게', '내요', '내일', '너', '너희', '너희들', '네', '넷', '네요', '넷째', '년', '논하지 않다', '누가', '누가 알겠는가', '누구', '누군가', '눔', '느낌', '니다', '니스', '니

화장품마다 리뷰 처리

In [10]:
def pretreatment(reviews):
    okt = Okt()

    def process_review(text):
        pos_words = okt.pos(text)

        # 명사와 형용사 추출 (한 글자 명사 제외)
        nouns_and_adjectives = [
            word for word, pos in pos_words
            if (pos == 'Noun' and len(word) > 1) #or (pos == 'Adjective'  and len(word) > 1)
        ]

        # 불용어 제거
        filtered_words = [word for word in nouns_and_adjectives if word not in stop_words]

        return filtered_words

    # 전체 리뷰에 대해 전처리 수행
    processed_reviews = [process_review(review) for review in reviews]

    return processed_reviews

# 함수 호출
processed_reviews = pretreatment(df['review'])

# # 결과 출력
# for i, processed_review in enumerate(processed_reviews):
#     print(f"리뷰 {i+1}: {processed_review}")

In [11]:
print(processed_reviews[0])

['테스트', '구매', '올인원', '쓰시', '선물', '무난', '향기도', '무난', '구매', '중이', '남자친구', '바르샤', '대요', '남성', '관리', '선물', '구매', '구입', '화장품', '무척', '기억', '구입', '무난', '기억', '저가', '행사', '구입', '남편', '선물', '토너', '구매', '부담', '자극', '보습', '나이', '남성', '올인원', '형도', '계세', '용감', '세지', '세트', '구매', '선물', '지인', '추천', '구매', '쓰시', '구매', '의사', '선물', '무난', '쓰기', '향기', '매장', '자극', '무난', '화장품', '자극', '오빠', '사려', '구매', '무난', '전형', '님자', '할인', '가성', '분기', '분기', '남친', '생물', '구매', '용감', '추천', '복합', '자극', '올인원', '지도', '백화점', '화장품', '주로', '남성', '무난', '쓰기', '대서', '보습', '형도', '무난', '쓰기', '조항', '장벽', '보호', '템닙', '쓰기', '추천', '선물', '구매', '구매', '백화점', '브랜드', '옴므', '대요', '가성', '비도', '마음', '남자친구', '남자친구', '선물', '구매', '남친', '마음', '기분', '선물', '우루스', '적임', '흡수', '지성', '오빠', '무난', '기존', '적임', '별로', '적임', '무난', '계세', '사서', '가성', '마무리', '쓰기', '추천', '만해', '추천', '구매', '무난', '쓰기', '무난', '외삼촌', '선물', '구매', '테스트', '구매', '발라', '선물', '자극', '거부', '무난', '추천', '남성', '화장품', '매장', '테스트', '구입', '선물', '자극', '남자친구', '분기', '별로', '자주', '선물', '드릴', '드릴', '자극', '

In [12]:
len(processed_reviews[0])

203

Word2vec

In [ ]:
'''
vector_size >> 단어가 임베딩되는 벡터의 차원 수 (기본값 100)
window >> 양쪽으로 몇개의 단어를 고려할것인지
min_count >> 최소 몇번 이상 등장한 단어만 고려할것인지
workers >> 학습을 위한 프로세스 수
sg >> skip-gram(1) or CBOW(0)
'''

In [27]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from konlpy.tag import Okt
import pandas as pd

w2v_model = Word2Vec(sentences=processed_reviews, vector_size=100, window=50, min_count=1, workers=4, sg=0)

# 단어 벡터 추출
word_vectors = w2v_model.wv
words = list(word_vectors.index_to_key)

In [28]:
#Kmeans 군집화
n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
word_vectors_list = [word_vectors[word] for word in words]
kmeans.fit(word_vectors.vectors)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=7, random_state=42)

In [29]:
# 군집화 결과 출력
clusters = {}
for word, cluster in zip(words, kmeans.labels_):
    if cluster not in clusters:
        clusters[cluster] = []
    clusters[cluster].append(word)

# 각 군집에 속한 단어 출력
for cluster, words in clusters.items():
    print(f"군집 {cluster + 1}: {', '.join(words)}")

군집 5: 구매, 보습, 자극, 선물, 추천, 흡수, 수분, 건성, 가격, 얼굴, 발라, 여드름, 성분, 화장품, 화장, 타입, 냄새, 에센스, 세트, 시리즈, 비건, 오디너리, 레티놀
군집 2: 올인원, 쓰기, 남자친구, 진정, 적임, 트러블, 남편, 지성, 겨울, 용량, 케어, 발림, 남성, 마음, 거리, 토너, 부담, 구입, 기초, 후기, 기름, 할인, 살짝, 배송, 주문, 수분크림, 최고, 브랜드, 용감, 대요, 건조, 도움, 저녁, 기분, 기획, 구성, 앰플, 판매, 바디, 트리
군집 7: 무난, 복합, 가성, 마무리, 자주, 부지, 일리, 신랑, 분기, 남동생, 별로, 의사, 민감, 닥터, 동생, 리뷰, 매장, 만족, 역시, 남친, 사서, 정착, 성도, 통째, 스킨로션, 하나로, 예정, 금방, 행사, 세안, 거나, 미백, 여름철, 테스트, 워낙, 유수, 기도, 지속, 개선, 이드, 계절, 장벽, 크게, 여행, 패키지, 펌프, 조절, 피지, 주름, 샘플, 밀크, 미리, 걱정, 지원, 메이크업, 상태, 미스트, 오스, 단점, 파우더, 먼저, 카밍
군집 3: 무향, 아들, 지도, 지인, 발리, 밸런스, 달라, 해결, 중이, 추가, 관리, 대비, 강추, 유지, 기존, 기름기, 사계절, 겨울철, 촉촉, 굿굿, 쓰시, 안나, 아크, 포맨, 용기, 지성인, 레이어, 가을, 듭니, 날씨, 기본, 기간, 플루, 단계, 위해, 이유, 상큼, 퓨전, 포장, 가끔, 탄력, 기대, 디자인, 좁쌀, 각질, 아쿠아, 여러, 모공, 우르, 단독, 선택, 비도, 쫀쫀, 면도, 필요, 피부관리, 눅스, 레몬, 단종, 무조건, 개인, 라보, 생일, 제공, 선호, 선크림, 이신, 사드, 튜브, 사이즈, 밀리, 참고, 시작, 나이아신, 컨디션, 몇번, 찰떡, 아이템, 아마이드, 오프라인, 보호, 흉터, 분사, 형태, 마스크, 자연, 추출, 광고, 가루, 스푼, 레티노
군집 4: 독도, 양도, 오빠, 두번째, 라운드, 리뉴얼, 거만, 소나무, 군대, 듬뿍, 부모님, 호불호, 생신, 내내,

CountVectorizer

In [ ]:
corpus = df['review'].tolist()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

for i, review in enumerate(processed_reviews):
    cvect = CountVectorizer(ngram_range=(2,4),min_df=0.03, max_df=0.90, stop_words = stop_words)
    dtm = cvect.fit_transform(corpus)
    vocab = cvect.get_feature_names_out()

    w2v_model = Word2Vec(sentences=processed_reviews, vector_size=100, window=50, min_count=1, workers=4, sg=0)

    # 단어 벡터 추출
    word_vectors = w2v_model.wv
    words = list(word_vectors.index_to_key)

    #Kmeans 군집화
    n_clusters = 7
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    word_vectors_list = [word_vectors[word] for word in words]
    kmeans.fit(word_vectors.vectors)

    # 군집화 결과 출력
    clusters = {}
    for word, cluster in zip(words, kmeans.labels_):
        if cluster not in clusters:
            clusters[cluster] = []
        clusters[cluster].append(word)

    # 각 군집에 속한 단어 출력
    for cluster, words in clusters.items():
        print(f"군집 {cluster + 1}: {', '.join(words)}")
    print("--" * 100)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


군집 5: 구매, 보습, 올인원, 자극, 좋은, 선물, 추천, 흡수, 좋고, 수분, 건성, 가격, 얼굴, 트러블, 발라, 좋네요, 성분, 겨울, 화장품, 용량, 화장, 케어, 발림, 타입, 마음, 냄새, 후기, 할인, 에센스, 오디너리, 레티놀
군집 2: 쓰기, 무난, 순하고, 남자친구, 진정, 적임, 좋다고, 남편, 지성, 촉촉하고, 여드름, 없어서, 같습니다, 가볍게, 남성, 복합, 가성, 마무리, 거리, 토너, 편하고, 부담, 구입, 만족합니다, 기초, 자주, 기름, 부지, 일리, 같아서, 살짝, 배송, 건조한, 없어요, 주문, 좋았어요, 수분크림, 최고, 확실히, 분기, 브랜드, 있는데, 용감, 건조, 세트, 저녁, 리뷰, 기분, 시리즈, 트리, 미백
군집 6: 순해서, 신랑, 없는, 순한, 괜찮은, 가벼운, 촉촉해서, 남동생, 별로, 대요, 의사, 도움, 가볍고, 편하게, 촉촉하게, 민감, 동생, 매장, 무향, 역시, 만족, 남친, 사서, 정착, 성도, 촉촉한, 구성, 기획, 민감한, 예민한, 많아서, 저렴하게, 통째, 산뜻한, 촉촉해요, 스킨로션, 비건, 빠르고, 산뜻하게, 앰플, 건조해서, 하나로, 금방, 판매, 행사, 부드럽고, 바디, 있을, 좋음, 세안, 그런지, 거나, 부드럽게, 여름철, 테스트, 워낙, 유수, 기도, 지속, 개선, 관리, 여행, 장벽, 크게, 이드, 같은데, 패키지, 조절, 펌프, 기존, 주름, 피지, 밀크, 샘플, 걱정, 미리, 지원, 메이크업, 상태, 카밍
군집 3: 닥터, 아들, 좋아용, 예정, 지도, 순해요, 가벼워서, 좋아해요, 지인, 밸런스, 발리, 같네요, 양도, 독도, 편해요, 적당히, 간편하게, 중이, 달라, 해결, 계절, 추가, 좋았습니다, 대비, 강추, 없어, 괜찮아요, 빠르게, 무겁지, 유지, 사계절, 묽은, 산뜻하고, 기름기, 촉촉, 겨울철, 쓰시, 굿굿, 강하지, 많고, 나쁘지, 아크, 안나, 딱히, 같고, 괜찮아서, 촉촉하니, 듭니, 만족해요, 레이어, 솔직한, 용기, 가을, 지성인, 포맨, 적당한

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['귀차', '되어야', '말해', '싶다', '싶어', '싶은', '않는다', '않았다', '있었다', '화장품'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


군집 5: 구매, 보습, 올인원, 자극, 좋은, 선물, 추천, 쓰기, 흡수, 수분, 건성, 가격, 적임, 얼굴, 트러블, 발라, 좋네요, 성분, 화장품, 용량, 화장, 케어, 타입, 마음, 마무리, 기초, 후기, 할인, 에센스, 오디너리, 레티놀
군집 2: 무난, 좋고, 순하고, 남자친구, 진정, 좋다고, 남편, 지성, 촉촉하고, 여드름, 겨울, 없어서, 발림, 같습니다, 남성, 복합, 가성, 거리, 토너, 편하고, 부담, 구입, 냄새, 자주, 기름, 부지, 일리, 같아서, 살짝, 배송, 건조한, 주문, 좋았어요, 수분크림, 최고, 확실히, 분기, 브랜드, 있는데, 용감, 건조, 세트, 저녁, 리뷰, 기분, 시리즈
군집 7: 순해서, 가볍게, 만족합니다, 신랑, 없어요, 없는, 순한, 괜찮은, 가벼운, 촉촉해서, 남동생, 별로, 대요, 의사, 도움, 가볍고, 편하게, 촉촉하게, 민감, 동생, 매장, 무향, 역시, 만족, 남친, 사서, 정착, 성도, 촉촉한, 구성, 기획, 민감한, 예민한, 많아서, 저렴하게, 통째, 산뜻한, 촉촉해요, 스킨로션, 비건, 빠르고, 산뜻하게, 앰플, 건조해서, 하나로, 금방, 판매, 행사, 부드럽고, 바디, 있을, 좋음, 트리, 세안, 그런지, 거나, 미백, 여름철, 테스트, 워낙, 유수, 기도, 지속, 개선, 관리, 여행, 크게, 이드, 같은데, 패키지, 조절, 펌프, 주름, 밀크, 샘플, 미리, 메이크업, 상태, 카밍
군집 3: 닥터, 아들, 좋아용, 예정, 지도, 순해요, 가벼워서, 좋아해요, 부드럽게, 지인, 밸런스, 발리, 같네요, 양도, 독도, 편해요, 적당히, 간편하게, 중이, 달라, 해결, 계절, 추가, 좋았습니다, 대비, 장벽, 강추, 없어, 괜찮아요, 빠르게, 무겁지, 유지, 사계절, 묽은, 산뜻하고, 기름기, 촉촉, 겨울철, 기존, 쓰시, 굿굿, 강하지, 많고, 피지, 나쁘지, 아크, 안나, 딱히, 걱정, 같고, 지원, 괜찮아서, 촉촉하니, 듭니, 만족해요, 레이어, 솔직한, 용기, 가을, 지성인, 포맨, 적당한

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['귀차', '되어야', '말해', '싶다', '싶어', '싶은', '않는다', '않았다', '있었다', '화장품'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

군집 5: 구매, 보습, 올인원, 자극, 좋은, 선물, 추천, 흡수, 수분, 가격, 적임, 얼굴, 발라, 성분, 화장품, 용량, 화장, 타입, 후기, 에센스, 오디너리, 레티놀
군집 4: 쓰기, 무난, 좋고, 순하고, 건성, 남자친구, 진정, 좋다고, 트러블, 남편, 지성, 좋네요, 촉촉하고, 여드름, 겨울, 없어서, 케어, 발림, 같습니다, 남성, 마음, 복합, 가성, 마무리, 거리, 토너, 편하고, 부담, 구입, 냄새, 기초, 자주, 기름, 할인, 부지, 일리, 같아서, 살짝, 배송, 없어요, 주문, 좋았어요, 수분크림, 최고, 확실히, 분기, 브랜드, 용감, 건조, 세트, 저녁, 리뷰, 기분, 시리즈
군집 2: 순해서, 가볍게, 만족합니다, 건조한, 신랑, 없는, 순한, 괜찮은, 가벼운, 촉촉해서, 있는데, 남동생, 별로, 대요, 의사, 도움, 가볍고, 편하게, 촉촉하게, 민감, 동생, 매장, 무향, 역시, 만족, 남친, 사서, 정착, 성도, 촉촉한, 구성, 기획, 민감한, 많아서, 저렴하게, 통째, 산뜻한, 촉촉해요, 스킨로션, 비건, 빠르고, 산뜻하게, 앰플, 하나로, 금방, 판매, 행사, 부드럽고, 바디, 있을, 좋음, 트리, 세안, 그런지, 미백, 여름철, 테스트, 워낙, 유수, 기도, 지속, 개선, 관리, 여행, 크게, 이드, 같은데, 패키지, 조절, 펌프, 주름, 밀크, 샘플, 미리, 메이크업, 상태
군집 7: 닥터, 예민한, 아들, 좋아용, 건조해서, 예정, 지도, 가벼워서, 거나, 좋아해요, 부드럽게, 지인, 밸런스, 발리, 같네요, 양도, 편해요, 적당히, 간편하게, 중이, 달라, 해결, 계절, 추가, 좋았습니다, 대비, 장벽, 강추, 없어, 괜찮아요, 빠르게, 무겁지, 유지, 사계절, 묽은, 산뜻하고, 기름기, 촉촉, 겨울철, 기존, 쓰시, 굿굿, 강하지, 많고, 피지, 나쁘지, 아크, 안나, 딱히, 걱정, 같고, 지원, 촉촉하니, 듭니, 만족해요, 레이어, 솔직한, 용기, 가을, 지성인, 포맨, 적당한, 날씨, 오빠, 좋을것, 간편하고

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['귀차', '되어야', '말해', '싶다', '싶어', '싶은', '않는다', '않았다', '있었다', '화장품'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


군집 4: 구매, 보습, 올인원, 자극, 좋은, 선물, 추천, 흡수, 수분, 가격, 얼굴, 발라, 성분, 화장품, 용량, 화장, 후기, 에센스
군집 6: 쓰기, 무난, 좋고, 순하고, 건성, 남자친구, 진정, 적임, 좋다고, 트러블, 남편, 지성, 좋네요, 촉촉하고, 여드름, 겨울, 없어서, 케어, 발림, 같습니다, 타입, 남성, 마음, 복합, 마무리, 거리, 토너, 편하고, 부담, 구입, 냄새, 기초, 자주, 기름, 할인, 부지, 일리, 같아서, 살짝, 배송, 주문, 좋았어요, 수분크림, 최고, 확실히, 분기, 브랜드, 용감, 건조, 세트, 저녁, 리뷰, 기분, 시리즈, 오디너리, 레티놀
군집 2: 순해서, 가볍게, 가성, 만족합니다, 건조한, 신랑, 없어요, 없는, 순한, 괜찮은, 가벼운, 촉촉해서, 있는데, 남동생, 별로, 대요, 의사, 도움, 가볍고, 편하게, 촉촉하게, 민감, 동생, 매장, 무향, 역시, 만족, 남친, 사서, 정착, 성도, 촉촉한, 구성, 기획, 민감한, 많아서, 저렴하게, 통째, 산뜻한, 촉촉해요, 스킨로션, 비건, 빠르고, 산뜻하게, 앰플, 하나로, 금방, 판매, 행사, 부드럽고, 바디, 있을, 좋음, 트리, 세안, 그런지, 미백, 여름철, 테스트, 워낙, 유수, 기도, 지속, 개선, 관리, 여행, 장벽, 크게, 이드, 같은데, 패키지, 조절, 펌프, 기존, 주름, 밀크, 미리, 용기, 메이크업, 상태
군집 7: 닥터, 예민한, 아들, 좋아용, 건조해서, 예정, 지도, 가벼워서, 거나, 좋아해요, 부드럽게, 지인, 밸런스, 발리, 같네요, 양도, 독도, 편해요, 적당히, 간편하게, 중이, 달라, 해결, 계절, 추가, 좋았습니다, 강추, 없어, 괜찮아요, 빠르게, 무겁지, 유지, 사계절, 묽은, 산뜻하고, 기름기, 촉촉, 겨울철, 쓰시, 강하지, 많고, 피지, 나쁘지, 아크, 샘플, 안나, 딱히, 걱정, 같고, 지원, 촉촉하니, 듭니, 만족해요, 레이어, 솔직한, 가을, 지성인, 포맨, 적당한, 날씨, 오빠, 좋을것, 간편하고, 싫어

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['귀차', '되어야', '말해', '싶다', '싶어', '싶은', '않는다', '않았다', '있었다', '화장품'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

단어의 범위가 너무 넓어서 군집분석이나 lda는 적합하지 않은것 같음

화장품의 특징 하나를 제시하여 유사한 단어 추출

- '제형'과 비슷한 단어 추출

In [ ]:
# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences=processed_reviews, vector_size=100, window=50, min_count=1, workers=4, sg=1)

# '제형'과 유사한 단어 추출
similar_words = w2v_model.wv.most_similar('사용', topn=10)

# 유사한 단어 출력
print("사용과 유사한 단어들:")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

KeyError: "Key '사용' not present in vocabulary"

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences=processed_reviews, vector_size=100, window=20, min_count=1, workers=4, sg=1)

# 모델 어휘에 포함된 단어들 확인
vocab = list(w2v_model.wv.index_to_key)
print("어휘에 포함된 단어 수:", len(vocab))
if '촉촉' in vocab:
    print("'제형' 단어가 어휘에 포함되어 있습니다.")

# '제형'과 유사한 단어를 코사인 유사도를 통해 찾기
if '촉촉' in vocab:
    target_vector = w2v_model.wv['촉촉'].reshape(1, -1)
    similarities = {}

    for word in vocab:
        word_vector = w2v_model.wv[word].reshape(1, -1)
        similarity = cosine_similarity(target_vector, word_vector)[0][0]
        similarities[word] = similarity

    # 유사도를 기준으로 상위 10개 단어 추출
    similar_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:10]

    # 유사한 단어 출력
    print("촉촉과 유사한 단어들:")
    for word, similarity in similar_words:
        print(f"{word}: {similarity:.4f}")
else:
    print("'촉촉' 단어가 데이터에 포함되어 있지 않아 유사 단어를 추출할 수 없습니다.")

어휘에 포함된 단어 수: 5652
'제형' 단어가 어휘에 포함되어 있습니다.
촉촉과 유사한 단어들:
촉촉: 1.0000
불편함: 0.9909
남녀: 0.9900
쫀득: 0.9888
바르게: 0.9884
종류: 0.9868
꾸준하게: 0.9866
산뜻하고: 0.9865
형도: 0.9862
부드러워서: 0.9856
